In [389]:
import gym
import numpy as np 
import random 
import tensorflow as tf

In [390]:
actions = []
rewards = []
env = gym.make('CartPole-v0')
env.reset()
while True:
    action = env.action_space.sample()
    actions.append(action)
    state, reward, done, _ = env.step(action)
    rewards.append(state)
    if done:
        break
print ("Actions")        
print (actions)        
print ("State")
print (state)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Actions
[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
State
[-0.15101574 -1.3765592   0.23535674  2.25446843]


In [391]:
class Qlearning:
    def __init__(self,action_size=2,state_size=4,hidden_layers=10,name='Network'):
        self.action_size = action_size
        self.hidden_layers = hidden_layers
        self.state_size = state_size
        self.name = name
    
    def net(self):
        with tf.variable_scope(self.name):
            self.inputs = tf.placeholder(tf.float32,[None,self.state_size],name='inputs')
            self.actions = tf.placeholder(tf.int32,[None],name='actions')
            self.one_hot = tf.one_hot(self.actions,self.action_size)
            
            self.fc1 = tf.contrib.layers.fully_connected(self.inputs,self.hidden_layers)
            self.fc2 = tf.contrib.layers.fully_connected(self.fc1,self.hidden_layers)
            
            self.output = tf.contrib.layers.fully_connected(self.fc2,self.action_size)
            
            self.target_qs = tf.placeholder(tf.float32,[None],name='targets')
        
            self.Qs = tf.reduce_sum(tf.multiply(self.output,self.one_hot),axis=1)
            self.loss = tf.reduce_mean(tf.square(self.Qs - self.target_qs))
            self.opt = tf.train.AdamOptimizer(0.0002).minimize(self.loss)
            self.model_vars = tf.trainable_variables()    

In [392]:
from collections import deque

class Memory():
    def __init__(self,max_size):
        self.buffer = deque(maxlen=max_size)
    
    def append (self,experience):
    
        self.buffer.append(experience)
        
    def sample (self,batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)),
                               size = batch_size,
                               replace = False)
        return [self.buffer[ii] for ii in idx]
    

In [399]:
class training():
    def __init__(self):
        self.max_steps = 200
        self.max_episodes = 1000
        self.gamma = 0.99
        self.explore_start = 1.0
        self.explore_stop = 0.01
        self.decay_rate = 0.0001
        self.learning_rate = 0.0002
        self.pretrain_steps = 200
        self.memory_length = 10000
        self.batch_size = 200
        
    def train(self):
        tf.reset_default_graph()

        self.mainQn = Qlearning(name='main_network')
        #Initialize the networks 
        
        self.mainQn.net()
       
        memory = Memory(self.memory_length)
        ##Add Some experience in the Memory 
        env = gym.make('CartPole-v0')
        env.reset()
        t =0 
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
        while t < self.pretrain_steps:
            t +=1
            action = env.action_space.sample()
            next_state, reward, done, _ = env.step(action)
            if done:
                next_state = np.zeros(state.shape)
                memory.append((state,action,reward,next_state))
                env.reset()
                action = env.action_space.sample()
                state, reward, done, _ = env.step(action)
                
            else :
                memory.append((state,action,reward,next_state))
                state = next_state
        
        batch = memory.sample(self.batch_size)
        saver = tf.train.Saver()
        
        
        with tf.Session() as sess:
           
            sess.run(tf.initialize_all_variables())
            sess.run(tf.global_variables_initializer())
            reward_list = []
            steps = 0
            for episode in range(1,self.max_episodes):
                t = 0
                total_reward = 0 
                
                while t < self.max_steps:
                    steps += 1 
                    explore_p = self.explore_stop + (self.explore_start-self.explore_stop)*np.exp(-self.decay_rate*steps)
                    
                    
                    if explore_p < np.random.rand():
                        actions = env.action_space.sample()
                    else:
                       
                        feed= {self.mainQn.inputs : state.reshape((1,*state.shape))}

                        qlearning = sess.run(self.mainQn.output,feed_dict=feed)
                        action = np.argmax(qlearning)
                    next_state, reward, done, _ = env.step(action)
                    total_reward += reward
                    if done :
                        next_state = np.zeros(state.shape)
                        memory.append((state,action,reward,next_state))
                        t = self.max_steps
                        env.reset()
                        print ('Episode {}'.format(episode),
                                'Reward {}'.format(total_reward),
                                'Explore p {:.4f}'.format(explore_p))
                       
                        print (loss)
                        state, reward, done, _ = env.step(action)
                        
                    else:
                       
                        memory.append((state,action,reward,next_state))
                        state = next_state
                        steps +=1
                    
                    batch = memory.sample(self.batch_size)
                    
                    states = np.array([each[0] for each in batch])
                    actions = np.array([each[1] for each in batch])
                    reward = np.array([each[2] for each in batch])
                    next_state = np.array([each[3] for each in batch])
                   
                    target_Qs = sess.run(self.mainQn.output, feed_dict={self.mainQn.inputs: next_state})
                    episode_ends = (next_state == np.zeros(states[0].shape)).all(axis=1)
                    target_Qs[episode_ends] = (0, 0)
                    
                    targets = reward + self.gamma*np.max(target_Qs,axis=1)
                    loss,_ = sess.run([self.mainQn.loss,self.mainQn.opt],feed_dict={
                           self.mainQn.actions:actions,
                           self.mainQn.target_qs : targets,
                           self.mainQn.inputs : states                                                     
                    })
                       
                        
          
        

In [400]:
def main():
    flap = training()
    flap.train()


if __name__ == "__main__":
    main()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode 1 Reward 29.0 Explore p 0.9944
1.0610939
Episode 2 Reward 29.0 Explore p 0.9888
1.0930026
Episode 3 Reward 13.0 Explore p 0.9863
1.1258103
Episode 4 Reward 12.0 Explore p 0.9841
1.1349422
Episode 5 Reward 9.0 Explore p 0.9824
1.1353368
Episode 6 Reward 9.0 Explore p 0.9808
1.1771729
Episode 7 Reward 9.0 Explore p 0.9791
1.1721482
Episode 8 Reward 9.0 Explore p 0.9775
1.1976584
Episode 9 Reward 10.0 Explore p 0.9757
1.2014449
Episode 10 Reward 9.0 Explore p 0.9740
1.2440616
Episode 11 Reward 9.0 Explore p 0.9724
1.2188214
Episode 12 Reward 9.0 Explore p 0.9707
1.2508671
Episode 13 Reward 7.0 Explore p 0.9695
1.2793549
Episode 14 Reward 9.0 Explore p 0.9679
1.2417238
Episode 15 Reward 9.0 Explore p 0.9662
1.2997506
Episode 16 Reward 8.0 Explore p 0.9648
1.3250979
Episode 17 Reward 7.0 Explore p 0.9636
1.2937704
Episode 18 Reward 7.0 Explore p 0.9623
1.3327179
Episode 19 Reward 9.0 E

Episode 177 Reward 9.0 Explore p 0.7519
1.2263623
Episode 178 Reward 9.0 Explore p 0.7506
1.2466066
Episode 179 Reward 8.0 Explore p 0.7495
1.6902554
Episode 180 Reward 7.0 Explore p 0.7486
1.4894662
Episode 181 Reward 8.0 Explore p 0.7475
1.2564642
Episode 182 Reward 9.0 Explore p 0.7462
1.4756869
Episode 183 Reward 8.0 Explore p 0.7451
0.9395407
Episode 184 Reward 7.0 Explore p 0.7441
1.133925
Episode 185 Reward 9.0 Explore p 0.7429
1.3453505
Episode 186 Reward 9.0 Explore p 0.7417
1.2743334
Episode 187 Reward 9.0 Explore p 0.7404
0.82359177
Episode 188 Reward 8.0 Explore p 0.7393
2.1703043
Episode 189 Reward 8.0 Explore p 0.7382
1.0179741
Episode 190 Reward 8.0 Explore p 0.7371
0.8089009
Episode 191 Reward 8.0 Explore p 0.7360
1.3426731
Episode 192 Reward 7.0 Explore p 0.7351
1.0501809
Episode 193 Reward 9.0 Explore p 0.7339
1.5442266
Episode 194 Reward 11.0 Explore p 0.7323
1.303968
Episode 195 Reward 11.0 Explore p 0.7308
1.3396503
Episode 196 Reward 10.0 Explore p 0.7295
0.750601

Episode 344 Reward 36.0 Explore p 0.4924
3.7829833
Episode 345 Reward 36.0 Explore p 0.4890
3.3219733
Episode 346 Reward 21.0 Explore p 0.4870
3.6025553
Episode 347 Reward 46.0 Explore p 0.4827
3.5584867
Episode 348 Reward 28.0 Explore p 0.4801
1.907207
Episode 349 Reward 22.0 Explore p 0.4781
4.91177
Episode 350 Reward 11.0 Explore p 0.4771
4.2441597
Episode 351 Reward 30.0 Explore p 0.4743
6.174643
Episode 352 Reward 33.0 Explore p 0.4713
4.4822936
Episode 353 Reward 48.0 Explore p 0.4670
7.6630726
Episode 354 Reward 22.0 Explore p 0.4650
2.6242008
Episode 355 Reward 17.0 Explore p 0.4635
5.7001143
Episode 356 Reward 25.0 Explore p 0.4613
7.0666294
Episode 357 Reward 27.0 Explore p 0.4589
3.7762306
Episode 358 Reward 7.0 Explore p 0.4583
6.639585
Episode 359 Reward 27.0 Explore p 0.4559
4.570433
Episode 360 Reward 18.0 Explore p 0.4544
4.511942
Episode 361 Reward 21.0 Explore p 0.4526
5.0076427
Episode 362 Reward 29.0 Explore p 0.4501
6.831181
Episode 363 Reward 39.0 Explore p 0.4467

Episode 512 Reward 25.0 Explore p 0.2084
9.822816
Episode 513 Reward 22.0 Explore p 0.2075
12.424003
Episode 514 Reward 8.0 Explore p 0.2072
24.654549
Episode 515 Reward 9.0 Explore p 0.2069
10.6350355
Episode 516 Reward 27.0 Explore p 0.2058
7.565974
Episode 517 Reward 27.0 Explore p 0.2048
15.909126
Episode 518 Reward 8.0 Explore p 0.2045
37.42971
Episode 519 Reward 22.0 Explore p 0.2037
32.600075
Episode 520 Reward 9.0 Explore p 0.2034
16.105705
Episode 521 Reward 8.0 Explore p 0.2031
18.241112
Episode 522 Reward 14.0 Explore p 0.2025
18.943361
Episode 523 Reward 16.0 Explore p 0.2019
14.240508
Episode 524 Reward 21.0 Explore p 0.2012
16.186588
Episode 525 Reward 13.0 Explore p 0.2007
19.848269
Episode 526 Reward 32.0 Explore p 0.1995
18.083866
Episode 527 Reward 8.0 Explore p 0.1992
18.878904
Episode 528 Reward 8.0 Explore p 0.1989
27.330381
Episode 529 Reward 17.0 Explore p 0.1983
21.610998
Episode 530 Reward 29.0 Explore p 0.1972
19.967304
Episode 531 Reward 8.0 Explore p 0.1969


Episode 675 Reward 24.0 Explore p 0.1256
9.376301
Episode 676 Reward 27.0 Explore p 0.1250
20.353386
Episode 677 Reward 11.0 Explore p 0.1247
15.842703
Episode 678 Reward 18.0 Explore p 0.1243
18.910372
Episode 679 Reward 21.0 Explore p 0.1238
14.062461
Episode 680 Reward 9.0 Explore p 0.1236
25.004175
Episode 681 Reward 17.0 Explore p 0.1233
25.927858
Episode 682 Reward 14.0 Explore p 0.1230
15.987923
Episode 683 Reward 30.0 Explore p 0.1223
20.948952
Episode 684 Reward 8.0 Explore p 0.1221
20.078955
Episode 685 Reward 23.0 Explore p 0.1216
24.54081
Episode 686 Reward 19.0 Explore p 0.1212
12.402172
Episode 687 Reward 9.0 Explore p 0.1210
29.888828
Episode 688 Reward 8.0 Explore p 0.1209
27.929188
Episode 689 Reward 9.0 Explore p 0.1207
20.771273
Episode 690 Reward 9.0 Explore p 0.1205
48.32798
Episode 691 Reward 7.0 Explore p 0.1203
27.019112
Episode 692 Reward 14.0 Explore p 0.1200
11.231563
Episode 693 Reward 8.0 Explore p 0.1199
30.56835
Episode 694 Reward 9.0 Explore p 0.1197
26.

Episode 844 Reward 16.0 Explore p 0.0841
52.058594
Episode 845 Reward 8.0 Explore p 0.0840
24.064049
Episode 846 Reward 8.0 Explore p 0.0838
66.31595
Episode 847 Reward 14.0 Explore p 0.0836
47.56717
Episode 848 Reward 8.0 Explore p 0.0835
62.229668
Episode 849 Reward 31.0 Explore p 0.0831
18.203608
Episode 850 Reward 23.0 Explore p 0.0828
54.26669
Episode 851 Reward 8.0 Explore p 0.0826
89.583496
Episode 852 Reward 7.0 Explore p 0.0826
39.720936
Episode 853 Reward 9.0 Explore p 0.0824
26.745893
Episode 854 Reward 10.0 Explore p 0.0823
59.690987
Episode 855 Reward 9.0 Explore p 0.0822
23.74491
Episode 856 Reward 9.0 Explore p 0.0820
49.98731
Episode 857 Reward 9.0 Explore p 0.0819
25.397053
Episode 858 Reward 9.0 Explore p 0.0818
39.077633
Episode 859 Reward 20.0 Explore p 0.0815
37.498108
Episode 860 Reward 8.0 Explore p 0.0814
36.749428
Episode 861 Reward 9.0 Explore p 0.0813
39.75356
Episode 862 Reward 24.0 Explore p 0.0810
29.250175
Episode 863 Reward 32.0 Explore p 0.0805
44.12454